# 1. Introdução

O código deste Notebook realiza a extração de dados de um banco de dados criado e aplica um processo de ETL (Extração, Transformação e Carregamento) com o objetivo de gerar valor a partir das informações dos usuários. A finalidade principal é fornecer mensagens personalizadas de investimento, oferecendo insights financeiros relevantes e estratégias de economia adaptadas à situação de cada usuário.

# 2. Preparação dos Dados

O processo de preparação dos dados envolve a criação de um banco de dados e a inserção de dados fictícios nas tabelas correspondentes, representando usuários, contas e cartões. Essa etapa simula um ambiente realista, permitindo a execução das operações de Extração, Transformação e Carregamento (ETL) em dados semelhantes aos reais. Isso viabiliza a geração de mensagens personalizadas de investimento com base em métricas financeiras calculadas a partir dos dados inseridos, oferecendo insights valiosos aos usuários.

## 2.1. SQLite

O Sistema de Gerenciamento de Banco de Dados (SGBD) escolhido foi o SQLite devido à sua leveza e versatilidade para armazenar, gerenciar e recuperar dados. Sua natureza autônoma elimina a necessidade de um servidor externo. Com suporte a consultas SQL, índices e transações, o SQLite atende às demandas do desafio, oferecendo eficiência no gerenciamento de dados locais.

In [1]:
import sqlite3

## 2.2. Criação das Tabelas

O processo de criação das tabelas no banco de dados segue um fluxo bem definido. Primeiramente, são elaboradas instruções SQL que especificam a estrutura das tabelas, então é estabelecida uma conexão com o banco de dados. Utilizando essa conexão, as tabelas são criadas no banco de dados por meio da execução das instruções.

### 2.2.1. Tabela: Users

A tabela `users` armazena informações básicas sobre os usuários, como um identificador único (id) e o nome do usuário. Esta tabela é a base de referência para as outras tabelas, permitindo a associação de informações individuais com cada usuário.

In [2]:
create_table_users = '''
    CREATE TABLE users (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name VARCHAR(50)
    );
'''

### 2.2.2. Tabela: Accounts

A tabela `accounts` contém detalhes das contas financeiras dos usuários. Isso inclui informações como o número da conta, a agência bancária, o saldo da conta e o ID do usuário ao qual a conta pertence. A coluna `user_id` estabelece uma relação entre a conta e o usuário correspondente na tabela `users`.

In [3]:
create_table_accounts = '''
    CREATE TABLE accounts (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        number VARCHAR(6) UNIQUE,
        agency VARCHAR(4),
        balance REAL,
        user_id INTEGER,
        FOREIGN KEY (user_id) REFERENCES users(id)
    );
'''

### 2.2.3. Tabela: Cards

A tabela `cards` armazena dados relacionados aos cartões de crédito dos usuários. Isso inclui o número do cartão, a utilização atual do cartão, o limite do cartão e o ID do usuário associado. A coluna `account_id` estabelece uma relação entre o cartão e a conta correspondente na tabela `accounts`.

In [4]:
create_table_cards = '''
    CREATE TABLE cards (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        number VARCHAR(16) UNIQUE,
        usage REAL,
        card_limit REAL,
        account_id INTEGER,
        FOREIGN KEY (account_id) REFERENCES accounts(id)
    );
'''

### 2.2.4. Tabela: News

A tabela `news` armazena mensagens que serão entregues aos usuários. Cada registro na tabela inclui um ID exclusivo, o ID do usuário associado à mensagem e o conteúdo da mensagem em si.

In [5]:
create_table_news = '''
    CREATE TABLE news (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        message TEXT,
        user_id INTEGER,
        FOREIGN KEY (user_id) REFERENCES users(id)
    );
'''

### 2.2.5. Criando as Tabelas no Banco

Uma vez que as instruções SQL estejam definidas, é realizada a conexão com o banco de dados `SDW2023.db` e a criação das tabelas, seguindo as instruções predefinidas.

In [6]:
conn = sqlite3.connect('SDW2023.db')
cur = conn.cursor()
cur.execute(create_table_users)
cur.execute(create_table_accounts)
cur.execute(create_table_cards)
cur.execute(create_table_news)
conn.commit()
cur.close()
conn.close()

## 2.3. Inserção dos Dados

### 2.3.1. Dados

Nesta etapa, são definidos dados fictícios que simulam um ambiente real. Os dados são estruturados em listas de tuplas, onde cada lista representa uma tabela e cada tupla simula um registro correspondente. Cada tupla inclui informações pertinentes às entidades previamente definidas, como nomes de usuários, números de contas e números de cartões.

É importante ressaltar que, por enquanto, nenhuma informação será atribuída à tabela `news`. Os dados a serem inseridos nessa tabela serão tratados durante a fase de transformação dos dados.

A definição destes dados cobrem todas as variedades possíveis dentro deste contexto bancário:
- Usuário com uma conta e nenhum cartão;
- Usuário com uma conta e um ou dois cartões;
- Usuário com duas contas e nenhum, um ou dois cartões para cada conta.

In [7]:
users = [
    ('Maria Alice Santos',),
    ('José Silva',),
    ('Paulo Moreira',),
    ('Ana Rodrigues',),
    ('Carlos Lima',),
    ('Lúcia Almeida',),
    ('Fernando Sousa',),
    ('Mariana Costa',),
    ('Ronaldo Nunes',),
]

accounts = [
    ('000001', '0001', 100.0, 1),
    ('000002', '0002', 200.0, 2),
    ('000003', '0003', 300.0, 3),
    ('000004', '0003', 400.0, 3),
    ('000005', '0001', 500.0, 4),
    ('000006', '0001', 600.0, 4),
    ('000007', '0002', 700.0, 5),
    ('000008', '0002', 800.0, 5),
    ('000009', '0003', 900.0, 6),
    ('000010', '0004', 1000.0, 7),
    ('000011', '0004', 1100.0, 7),
    ('000012', '0002', 1200.0, 8),
    ('000013', '0002', 1300.0, 8),
    ('000014', '0001', 1400.0, 9),
    ('000015', '0001', 1500.0, 9),
]

cards = [
    ('**** **** **** 0001', 1000.0, 1000.0, 2),
    ('**** **** **** 0002', 1000.0, 1500.0, 5),
    ('**** **** **** 0003', 1000.0, 2000.0, 7),
    ('**** **** **** 0004', 1000.0, 2500.0, 8),
    ('**** **** **** 0005', 1000.0, 3000.0, 9),
    ('**** **** **** 0006', 500.0, 500.0, 9),
    ('**** **** **** 0007', 500.0, 1000.0, 10),
    ('**** **** **** 0008', 500.0, 1500.0, 10),
    ('**** **** **** 0009', 500.0, 2000.0, 11),
    ('**** **** **** 0010', 500.0, 2500.0, 11),
    ('**** **** **** 0011', 500.0, 3000.0, 12),
    ('**** **** **** 0012', 250.0, 250.0, 12),
    ('**** **** **** 0013', 250.0, 500.0, 13),
    ('**** **** **** 0014', 250.0, 750.0, 15),
    ('**** **** **** 0015', 250.0, 1000.0, 15),
]

### 2.3.2. Inserção dos Dados no Banco

Com os dados estruturados, é feita uma nova conexão com o banco. Utilizando um cursor, ele itera pelas listas de usuarios, contas e cartões, executando instruções SQL para inserir os dados definidos nas tabelas correspondentes, finalizando a etapa de preparação dos dados.

In [8]:
conn = sqlite3.connect('SDW2023.db')
cur = conn.cursor()

for user in users:
    cur.execute('INSERT INTO users (name) VALUES (?)', user)

for account in accounts:
    cur.execute('INSERT INTO accounts (number, agency, balance, user_id) VALUES (?, ?, ?, ?)', account)

for card in cards:
    cur.execute('INSERT INTO cards (number, usage, card_limit, account_id) VALUES (?, ?, ?, ?)', card)

conn.commit()
conn.close()

# 3. Extração, Transformação e Carregamento

Continua...